In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
AWS_ACCESS_KEY = os.getenv("ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("SECRET_KEY")

WEAVIATE_CLOUD_URL = os.getenv("WEAVIATE_CLOUD_URL")
WEAVIATE_CLOUD_KEY = os.getenv("WEAVIATE_CLOUD_KEY_READ")

## Connect

In [ ]:
import weaviate, json

# Connect to a Weaviate Cloud instance
client = weaviate.connect_to_wcs(
    cluster_url=WEAVIATE_CLOUD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_CLOUD_KEY),

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
    },
)

# Connect to the local instance deployed with Docker Compose
# client = weaviate.connect_to_local(
#     headers={
#         "X-AWS-Access-Key": AWS_ACCESS_KEY,
#         "X-AWS-Secret-Key": AWS_SECRET_KEY,
#     },
# )

client.is_ready()

## Data Preview

In [ ]:
# Get the collection
movies = client.collections.get("Movies")

response = movies.query.fetch_objects(limit=5)

for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        f"https://www.themoviedb.org/movie/{o.properties['tmdb_id']}",
        o.properties["poster_url"]
    )

## Title search

In [ ]:
import weaviate.classes.query as wq

movies = client.collections.get("Movies")

# Perform query
response = movies.query.near_text(
    query="A joyful holiday film",
    target_vector="title",  # The target vector to search against
    limit=5,
    # return_properties=["title", "release_date", "tmdb_id", "poster_url"],
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_url"]
    )

    # Print the distance of the object from the query
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

In [ ]:
import weaviate.classes.query as wq

# Get the collection
movies = client.collections.get("Movies")

# Perform query
response = movies.query.near_text(
    query="A joyful holiday film",
    target_vector="overview",  # The target vector to search against
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_url"]
    )

    # Print the distance of the object from the query
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

In [ ]:
client.close()